In [100]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [101]:
random_state = 59
np.random.seed(random_state)
torch.manual_seed(random_state)
if torch.cuda.is_available():
    torch.cuda.manual_seed(random_state)
    

In [102]:
device = torch.device('cude' if torch.cuda.is_available() else 'cpu')

In [103]:
data = pd.read_csv('Auto_MPG_data.csv')

In [104]:
X = data.drop(columns = 'MPG').values
y = data['MPG'].values

In [105]:
val_size = 0.2
test_size = 0.125
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size = val_size,
    random_state = random_state,
    shuffle = is_shuffle)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train,
    test_size = test_size,
    random_state = random_state,
    shuffle = is_shuffle)

In [106]:
normalizer = StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_val = normalizer.transform(X_val)
X_test = normalizer.transform(X_test)

X_train = torch.tensor(X_train, dtype = torch.float32)
X_val = torch.tensor(X_val, dtype = torch.float32)
X_test = torch.tensor(X_test, dtype = torch.float32)
y_train = torch.tensor(y_train, dtype = torch.float32)
y_val = torch.tensor(y_val, dtype = torch.float32)
y_test = torch.tensor(y_test, dtype = torch.float32)

In [107]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [108]:
batch_size = 32
train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True)
val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=False)

In [109]:
# class MLP(nn.Module):
#     def __init__(self, input_dims, hidden_dims, output_dims):
#         super().__init__()
#         self.linear1 = nn.Linear(input_dims, hidden_dims)
#         self.linear2 = nn.Linear(hidden_dims, hidden_dims)
#         self.output = nn.Linear(hidden_dims, output_dims)
        
#     def forward(self, x):
#         x = self.linear1(x)
#         x = F.relu(x)
#         x = self.linear2(x)
#         x = F.relu(x)
#         out = self.output(x)
#         return out.squeeze(1)

In [110]:
# Question 8.1 -> R2: 0.772823691368103
# class MLP(nn.Module):
#     def __init__(self, input_dims, hidden_dims, output_dims):
#         super().__init__()
#         self.linear = nn.Linear(input_dims, output_dims)

#     def forward(self, x):
#         out = self.linear(x)
#         return out.squeeze(1)

In [111]:
# Question 8.2 -> R2: 0.8610948324203491
# class MLP(nn.Module):
#     def __init__(self, input_dims, hidden_dims, output_dims):
#         super().__init__()
#         self.linear1 = nn.Linear(input_dims, hidden_dims)
#         self.linear2 = nn.Linear(hidden_dims, hidden_dims)
#         self.output = nn.Linear(hidden_dims, output_dims)
        
#     def forward(self, x):
#         x = self.linear1(x)
#         x = F.sigmoid(x)
#         x = self.linear2(x)
#         x = F.sigmoid(x)
#         out = self.output(x)
#         return out.squeeze(1)

In [ ]:
# Question 8.3 -> R2: 0.8503545522689819
class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims):
        super().__init__()
        self.linear1 = nn.Linear(input_dims, hidden_dims)
        self.linear2 = nn.Linear(hidden_dims, hidden_dims)
        self.output = nn.Linear(hidden_dims, output_dims)
        
    def forward(self, x):
        x = self.linear1(x)
        x = F.tanh(x)
        x = self.linear2(x)
        x = F.tanh(x)
        out = self.output(x)
        return out.squeeze(1)

In [113]:
input_dims = X_train.shape[1]
output_dims = 1
hidden_dims = 64

model = MLP(input_dims = input_dims,
            hidden_dims = hidden_dims,
            output_dims = output_dims).to(device)

In [114]:
lr = 1e-2
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = lr)

In [115]:
def r_squared(y_true, y_pred):
    y_true = torch.Tensor(y_true).to(device)
    y_pred = torch.Tensor(y_pred).to(device)
    mean_true = torch.mean(y_true)
    ss_tot = torch.sum((y_true - mean_true) ** 2)
    ss_res = torch.sum((y_true - y_pred) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    return r2

In [116]:
epochs = 100
train_losses = []
val_losses = []
train_r2 = []
val_r2 = []

for epoch in range(epochs):
    train_loss = 0.0
    train_target = []
    val_target = []
    train_predict = []
    val_predict = []
    model.train()
    for X_samples, y_samples in train_loader:
        X_samples = X_samples.to(device)
        y_samples = y_samples.to(device)
        optimizer.zero_grad()
        outputs = model(X_samples)
        train_predict += outputs.tolist()
        train_target += y_samples.tolist()
        loss = criterion(outputs, y_samples)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    train_r2.append(r_squared(train_target, train_predict))
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_samples, y_samples in val_loader:
            X_samples = X_samples.to(device)
            y_samples = y_samples.to(device)
            outputs = model(X_samples)
            val_predict += outputs.tolist()
            val_target += y_samples.tolist()
            loss = criterion(outputs, y_samples)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    val_r2.append(r_squared(val_target, val_predict))
    print(f'\nEPOCH {epoch + 1}:\t Training loss: {train_loss:.3f}\t Validation loss: {val_loss:.3f}')
            
            


EPOCH 1:	 Training loss: 241.367	 Validation loss: 16.744

EPOCH 2:	 Training loss: 25.802	 Validation loss: 10.408

EPOCH 3:	 Training loss: 17.005	 Validation loss: 8.828

EPOCH 4:	 Training loss: 12.898	 Validation loss: 7.989

EPOCH 5:	 Training loss: 12.065	 Validation loss: 7.318

EPOCH 6:	 Training loss: 10.612	 Validation loss: 6.559

EPOCH 7:	 Training loss: 9.599	 Validation loss: 6.742

EPOCH 8:	 Training loss: 9.545	 Validation loss: 8.186

EPOCH 9:	 Training loss: 9.627	 Validation loss: 11.149

EPOCH 10:	 Training loss: 10.421	 Validation loss: 12.589

EPOCH 11:	 Training loss: 10.581	 Validation loss: 6.085

EPOCH 12:	 Training loss: 8.204	 Validation loss: 5.374

EPOCH 13:	 Training loss: 8.724	 Validation loss: 6.749

EPOCH 14:	 Training loss: 8.068	 Validation loss: 5.731

EPOCH 15:	 Training loss: 7.931	 Validation loss: 6.713

EPOCH 16:	 Training loss: 8.309	 Validation loss: 6.800

EPOCH 17:	 Training loss: 7.815	 Validation loss: 6.303

EPOCH 18:	 Training loss: 

In [117]:
model.eval()
with torch.no_grad():
    y_hat = model(X_test)
    test_set_r2 = r_squared(y_hat, y_test)
    print('Evaluation on test set:')
    print(f'R2: {test_set_r2}')

Evaluation on test set:
R2: 0.8503545522689819
